In [18]:
"""
Generador de Dashboard HTML - Emergencias Colombia 1998-2024
Ejecutar: python generar_dashboard_html.py
Genera: dashboard_emergencias.html  (en la misma carpeta)
"""

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
import csv as _csv
import unicodedata
import warnings
warnings.filterwarnings("ignore")

# ── CARGAR DATOS ──────────────────────────────────────────────────────────────
FILE   = r"C:\Users\User\Downloads\CONSOLIDADO_EMERGENCIAS_1998-2024_v2.csv"
OUTPUT = r"C:\Users\User\Downloads\dashboard_emergencias.html"

print("Cargando datos...")
with open(FILE, "rb") as f:
    muestra = f.read(4000).decode("latin-1")
sep = _csv.Sniffer().sniff(muestra, delimiters=",;\t|").delimiter
print(f"  → Separador detectado: '{sep}'")

for enc in ["utf-8-sig", "utf-8", "latin-1"]:
    try:
        df = pd.read_csv(FILE, sep=sep, dtype=str, encoding=enc, on_bad_lines="skip")
        df.columns = df.columns.str.strip()
        cols_str = " ".join(df.columns)
        if "AÑO" in cols_str or "ANO" in cols_str or "AÃ" not in cols_str:
            print(f"  → Encoding usado: {enc}")
            break
    except Exception as e:
        print(f"  → {enc} falló: {e}")

print(f"  → {len(df):,} filas cargadas, {len(df.columns)} columnas")

# ── LIMPIAR ───────────────────────────────────────────────────────────────────
print("  → Columnas detectadas:", df.columns.tolist())

def _norm(s):
    return unicodedata.normalize("NFKD", str(s)).encode("ascii", "ignore").decode().strip().upper()

col_norm = {_norm(c): c for c in df.columns}

def _find(nombre):
    if nombre in df.columns: return nombre
    k = _norm(nombre)
    if k in col_norm: return col_norm[k]
    for orig_norm, orig in col_norm.items():
        if k in orig_norm or orig_norm in k:
            return orig
    return None

for estandar in ["AÑO", "MES", "DEPARTAMENTO", "MUNICIPIO", "EVENTO"]:
    real = _find(estandar)
    if real and real != estandar:
        df.rename(columns={real: estandar}, inplace=True)
        print(f"  → Renombrado: '{real}' → '{estandar}'")

NUM_COLS = ["MUERTOS", "HERIDOS", "DESAPA.", "PERSONAS", "FAMILIAS",
            "VIV.DESTRU.", "VIV.AVER.", "HECTAREAS", "RECURSOS EJECUTADOS"]
for c in NUM_COLS:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce").fillna(0)

for col in ["AÑO", "MES"]:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")

for col in ["DEPARTAMENTO", "MUNICIPIO", "EVENTO"]:
    if col in df.columns:
        df[col] = df[col].astype(str).str.strip().str.upper()

if "AÑO" not in df.columns:
    for c in df.columns:
        try:
            tmp = pd.to_numeric(df[c], errors="coerce")
            if tmp.between(1990, 2030).sum() > len(df) * 0.5:
                df.rename(columns={c: "AÑO"}, inplace=True)
                df["AÑO"] = tmp
                print(f"  → Columna año detectada automáticamente: '{c}'")
                break
        except:
            pass

if "AÑO" not in df.columns:
    raise ValueError("No se encontró columna de año. Columnas disponibles: " + str(df.columns.tolist()))

df = df[df["AÑO"].notna()].copy()
df["AÑO"] = df["AÑO"].astype(int)

MESES_NOMBRE = {1:"Ene",2:"Feb",3:"Mar",4:"Abr",5:"May",6:"Jun",
                7:"Jul",8:"Ago",9:"Sep",10:"Oct",11:"Nov",12:"Dic"}

# ── ESTILOS BASE — Paleta riesgo hídrico oscura ───────────────────────────────
DARK_BG  = "#050d14"       # fondo gráfica
PAPER    = "#07111a"       # fondo card
FONT_C   = "#d4e9f7"       # texto general
GRID_C   = "#0e2233"       # líneas de grilla
ACCENT1  = "#f5c518"       # amarillo neón (eventos)
ACCENT2  = "#e63946"       # rojo (fallecidos)
ACCENT3  = "#00e5a0"       # verde agua (personas)
ACCENT4  = "#3a9eca"       # azul claro (viviendas)
TITLE_C  = "#ffffff"       # títulos

base_layout = dict(
    paper_bgcolor=PAPER, plot_bgcolor=DARK_BG,
    font=dict(color=FONT_C, size=11, family="'Segoe UI', Arial, sans-serif"),
    margin=dict(l=50, r=20, t=55, b=50),
    xaxis=dict(gridcolor=GRID_C, zerolinecolor=GRID_C, tickfont=dict(color=FONT_C)),
    yaxis=dict(gridcolor=GRID_C, zerolinecolor=GRID_C, tickfont=dict(color=FONT_C)),
    title_font=dict(color=TITLE_C, size=13, family="'Segoe UI', Arial, sans-serif"),
)

# ── KPIs ──────────────────────────────────────────────────────────────────────
total_ev      = len(df)
total_muertos = int(df["MUERTOS"].sum())  if "MUERTOS"    in df.columns else 0
total_pers    = int(df["PERSONAS"].sum()) if "PERSONAS"   in df.columns else 0
vd = df["VIV.DESTRU."].sum() if "VIV.DESTRU." in df.columns else 0
va = df["VIV.AVER."].sum()   if "VIV.AVER."   in df.columns else 0
total_viv     = int(vd + va)
deptos_afect  = df["DEPARTAMENTO"].nunique() if "DEPARTAMENTO" in df.columns else 0

kpi_html = f"""
<div style="display:flex;gap:12px;flex-wrap:wrap;margin:20px 0;">
  <div style="flex:1;min-width:150px;background:#07111a;border-left:4px solid #f5c518;border-radius:4px;padding:14px 18px;">
    <div style="font-size:0.65rem;font-weight:700;letter-spacing:2px;color:#7aafc9;text-transform:uppercase;margin-bottom:6px;">EVENTOS</div>
    <div style="font-size:1.9rem;font-weight:900;color:#f5c518;line-height:1;">{total_ev:,}</div>
  </div>
  <div style="flex:1;min-width:150px;background:#07111a;border-left:4px solid #e63946;border-radius:4px;padding:14px 18px;">
    <div style="font-size:0.65rem;font-weight:700;letter-spacing:2px;color:#7aafc9;text-transform:uppercase;margin-bottom:6px;">FALLECIDOS</div>
    <div style="font-size:1.9rem;font-weight:900;color:#e63946;line-height:1;">{total_muertos:,}</div>
  </div>
  <div style="flex:1;min-width:150px;background:#07111a;border-left:4px solid #00e5a0;border-radius:4px;padding:14px 18px;">
    <div style="font-size:0.65rem;font-weight:700;letter-spacing:2px;color:#7aafc9;text-transform:uppercase;margin-bottom:6px;">PERSONAS AFECTADAS</div>
    <div style="font-size:1.9rem;font-weight:900;color:#00e5a0;line-height:1;">{total_pers:,}</div>
  </div>
  <div style="flex:1;min-width:150px;background:#07111a;border-left:4px solid #3a9eca;border-radius:4px;padding:14px 18px;">
    <div style="font-size:0.65rem;font-weight:700;letter-spacing:2px;color:#7aafc9;text-transform:uppercase;margin-bottom:6px;">VIVIENDAS AFECTADAS</div>
    <div style="font-size:1.9rem;font-weight:900;color:#3a9eca;line-height:1;">{total_viv:,}</div>
  </div>
  <div style="flex:1;min-width:150px;background:#07111a;border-left:4px solid #a78bfa;border-radius:4px;padding:14px 18px;">
    <div style="font-size:0.65rem;font-weight:700;letter-spacing:2px;color:#7aafc9;text-transform:uppercase;margin-bottom:6px;">DPTOS. AFECTADOS</div>
    <div style="font-size:1.9rem;font-weight:900;color:#a78bfa;line-height:1;">{deptos_afect:,}</div>
  </div>
</div>
"""

# ── G1: Área temporal ─────────────────────────────────────────────────────────
print("Generando gráficas...")
by_year = df.groupby("AÑO").size().reset_index(name="eventos")
fig1 = px.area(by_year, x="AÑO", y="eventos",
               title="📈 Eventos Registrados por Año",
               color_discrete_sequence=[ACCENT1])
fig1.update_layout(**base_layout)
fig1.update_traces(fillcolor="rgba(245,197,24,0.15)", line=dict(width=2.5, color=ACCENT1))

# ── G2: Donut tipo evento ─────────────────────────────────────────────────────
ev_counts = df["EVENTO"].value_counts().head(10).reset_index()
ev_counts.columns = ["EVENTO", "n"]
fig2 = px.pie(ev_counts, names="EVENTO", values="n",
              title="🔵 Distribución por Tipo de Evento",
              color_discrete_sequence=["#f5c518","#e63946","#00e5a0","#3a9eca","#a78bfa","#fb8c00","#26c6da","#ef5350","#66bb6a","#ab47bc"], hole=0.45)
fig2.update_layout(paper_bgcolor=PAPER, plot_bgcolor=DARK_BG, font=dict(color=FONT_C, family="'Inter', Arial"),
                   margin=dict(l=10, r=10, t=50, b=10), title_font=dict(color=TITLE_C, size=13))

# ── G3: Barras horizontales departamentos ─────────────────────────────────────
dep_ev = df.groupby("DEPARTAMENTO").size().nlargest(15).reset_index(name="n").sort_values("n")
fig3 = px.bar(dep_ev, x="n", y="DEPARTAMENTO", orientation="h",
              title="🏔️ Top 15 Departamentos con más Eventos",
              color="n", color_continuous_scale=[[0,"#0e2233"],[0.4,"#1a6a9a"],[1.0,"#f5c518"]])
fig3.update_layout(**base_layout, coloraxis_showscale=False, yaxis_title="")

# ── G4: Heatmap mes × año ─────────────────────────────────────────────────────
hm = df.groupby(["AÑO", "MES"]).size().reset_index(name="n")
hm_pivot = hm.pivot(index="MES", columns="AÑO", values="n").fillna(0)
hm_pivot.index = [MESES_NOMBRE.get(i, i) for i in hm_pivot.index]
fig4 = px.imshow(hm_pivot, color_continuous_scale=[[0,"#050d14"],[0.3,"#0a3a5c"],[0.65,"#1a7abf"],[1.0,"#f5c518"]],
                 title="🔥 Heatmap de Eventos: Mes vs Año",
                 labels=dict(color="Eventos"), aspect="auto")
fig4.update_layout(paper_bgcolor=PAPER, font=dict(color=FONT_C),
                   margin=dict(l=50, r=20, t=50, b=40))

# ── G5: Víctimas apiladas por año ─────────────────────────────────────────────
vcols = [c for c in ["MUERTOS", "HERIDOS", "DESAPA."] if c in df.columns]
vic_yr = df.groupby("AÑO")[vcols].sum().reset_index()
fig5 = go.Figure()
colors_vic = [ACCENT2, "#fb8c00", ACCENT1]
for i, col in enumerate(vcols):
    fig5.add_trace(go.Bar(x=vic_yr["AÑO"], y=vic_yr[col], name=col,
                          marker_color=colors_vic[i]))
fig5.update_layout(barmode="stack", title="💔 Víctimas por Año",
                   **base_layout, legend=dict(bgcolor="rgba(0,0,0,0)"))

# ── G6: Treemap personas por departamento ─────────────────────────────────────
tm = df.groupby("DEPARTAMENTO")["PERSONAS"].sum().nlargest(20).reset_index()
tm = tm[tm["PERSONAS"] > 0]
fig6 = px.treemap(tm, path=["DEPARTAMENTO"], values="PERSONAS",
                  title="🗺️ Personas Afectadas por Departamento (top 20)",
                  color="PERSONAS", color_continuous_scale=[[0,"#0a2a3e"],[0.5,"#1a6a9a"],[1.0,"#00e5a0"]])
fig6.update_layout(paper_bgcolor=PAPER, font=dict(color=FONT_C),
                   margin=dict(l=10, r=10, t=50, b=10))

# ── G7: Barras apiladas horizontales — Víctimas por Departamento ─────────────
vic_cols = [c for c in ["MUERTOS", "HERIDOS", "DESAPA."] if c in df.columns]
top10_dep = df["DEPARTAMENTO"].value_counts().head(10).index.tolist()
vic_dep = (
    df[df["DEPARTAMENTO"].isin(top10_dep)]
    .groupby("DEPARTAMENTO")[vic_cols].sum()
    .reset_index()
)
# Ordenar por total de víctimas
vic_dep["TOTAL"] = vic_dep[vic_cols].sum(axis=1)
vic_dep = vic_dep.sort_values("TOTAL")

fig7 = go.Figure()
colors_v = {"MUERTOS": ACCENT2, "HERIDOS": "#fb8c00", "DESAPA.": ACCENT1}
for col in vic_cols:
    fig7.add_trace(go.Bar(
        name=col,
        x=vic_dep[col],
        y=vic_dep["DEPARTAMENTO"],
        orientation="h",
        marker_color=colors_v.get(col, "#95a5a6"),
        hovertemplate="<b>%{y}</b><br>" + col + ": %{x:,}<extra></extra>",
    ))
fig7.update_layout(
    barmode="stack",
    title="🩸 Composición de Víctimas — Top 10 Departamentos",
    paper_bgcolor=PAPER, plot_bgcolor=DARK_BG,
    font=dict(color=FONT_C, size=11),
    margin=dict(l=10, r=20, t=50, b=40),
    xaxis=dict(title="Total Víctimas", gridcolor=GRID_C, zerolinecolor=GRID_C),
    yaxis=dict(title="", gridcolor=GRID_C, zerolinecolor=GRID_C),
    legend=dict(bgcolor="rgba(0,0,0,0)", orientation="h", y=1.08),
)

# ── G8: Estacionalidad mensual ────────────────────────────────────────────────
est = df.groupby("MES").size().reset_index(name="n")
est["mes_nom"] = est["MES"].map(MESES_NOMBRE)
fig8 = px.bar(est, x="mes_nom", y="n",
              title="📅 Estacionalidad: Total de Eventos por Mes",
              color="n", color_continuous_scale=[[0,"#0a1e2e"],[0.5,"#1a6aaa"],[1.0,"#3a9eca"]])
fig8.update_layout(**base_layout, coloraxis_showscale=False, xaxis_title="Mes")

# ── G9: Bubble Chart — Familias Afectadas vs Viviendas Destruidas ───────────
bbl_agg = {"eventos": ("DEPARTAMENTO", "count")}
for c, alias in [("FAMILIAS","familias"),("VIV.DESTRU.","viv_dest"),
                 ("VIV.AVER.","viv_aver"),("PERSONAS","personas"),("MUERTOS","muertos")]:
    if c in df.columns:
        bbl_agg[alias] = (c, "sum")

bbl = df.groupby("DEPARTAMENTO").agg(**bbl_agg).reset_index()
for col in ["familias","viv_dest","viv_aver","personas","muertos"]:
    if col not in bbl.columns: bbl[col] = 0

bbl = bbl[bbl["familias"] > 0].nlargest(22, "personas").copy()
bbl["viv_total"] = bbl["viv_dest"] + bbl["viv_aver"]
bbl["label_k"]   = bbl["personas"].apply(lambda v: f"{v/1000:.0f}k")

fig9 = go.Figure()
fig9.add_trace(go.Scatter(
    x=bbl["familias"],
    y=bbl["viv_total"],
    mode="markers+text",
    text=bbl["DEPARTAMENTO"].str.title(),
    textposition="top center",
    textfont=dict(size=8, color=FONT_C),
    marker=dict(
        size=bbl["personas"] / bbl["personas"].max() * 80 + 10,
        color=bbl["muertos"],
        colorscale=[[0,"#0a3a5c"],[0.5,"#e63946"],[1.0,"#f5c518"]],
        showscale=True,
        colorbar=dict(
            title=dict(text="Fallecidos", font=dict(color=FONT_C, size=10)),
            tickfont=dict(color=FONT_C, size=9),
            len=0.6, thickness=12,
        ),
        opacity=0.85,
        line=dict(width=1, color="#0f1923"),
    ),
    hovertemplate=(
        "<b>%{text}</b><br>"
        "Familias afectadas: %{x:,}<br>"
        "Viviendas afectadas: %{y:,}<extra></extra>"
    ),
    showlegend=False,
))
fig9.update_layout(
    title="🫧 Familias vs Viviendas Afectadas por Departamento<br>"
          "<sup>Tamaño = personas afectadas · Color = fallecidos</sup>",
    paper_bgcolor=PAPER, plot_bgcolor=DARK_BG,
    font=dict(color=FONT_C, size=11),
    margin=dict(l=20, r=80, t=70, b=50),
    xaxis=dict(title="Total Familias Afectadas", gridcolor=GRID_C, zerolinecolor=GRID_C),
    yaxis=dict(title="Total Viviendas Afectadas", gridcolor=GRID_C, zerolinecolor=GRID_C),
)

# ── G10: Personas afectadas por año (línea) ───────────────────────────────────
pers_yr = df.groupby("AÑO")["PERSONAS"].sum().reset_index()
fig10 = px.line(pers_yr, x="AÑO", y="PERSONAS",
                title="👥 Total Personas Afectadas por Año",
                markers=True, color_discrete_sequence=[ACCENT3])
fig10.update_layout(**base_layout)
fig10.update_traces(line=dict(width=2.5), marker=dict(size=6))

# ── ENSAMBLAR HTML ────────────────────────────────────────────────────────────
def fig_div(fig):
    return pio.to_html(fig, full_html=False, include_plotlyjs=False,
                       config={"responsive": True, "displayModeBar": False})

print("Ensamblando HTML...")

plotly_cdn = '<script src="https://cdn.plot.ly/plotly-2.27.0.min.js"></script>'

html_content = f"""<!DOCTYPE html>
<html lang="es">
<head>
<meta charset="UTF-8">
<meta name="viewport" content="width=device-width, initial-scale=1.0">
<title>Dashboard Emergencias Colombia 1998–2024</title>
{plotly_cdn}
<link href="https://fonts.googleapis.com/css2?family=Inter:wght@400;600;700;900&display=swap" rel="stylesheet">
<style>
  * {{ box-sizing: border-box; margin: 0; padding: 0; }}
  body {{
    font-family: 'Inter', 'Segoe UI', Arial, sans-serif;
    background: #020b12;
    color: #d4e9f7;
  }}
  header {{
    background: #07111a;
    padding: 22px 32px;
    border-bottom: 1px solid #0e2233;
    display: flex;
    align-items: flex-start;
    justify-content: space-between;
    flex-wrap: wrap;
    gap: 12px;
  }}
  .header-left h1 {{
    font-size: 1.45rem;
    font-weight: 900;
    color: #ffffff;
    letter-spacing: 0.5px;
    text-transform: uppercase;
  }}
  .header-left h1 span {{ color: #3a9eca; }}
  .header-left p {{
    font-size: 0.72rem;
    color: #4a7a99;
    margin-top: 5px;
    letter-spacing: 1.5px;
    text-transform: uppercase;
  }}
  .header-badge {{
    background: #0e2233;
    border: 1px solid #1a3a55;
    border-radius: 6px;
    padding: 10px 18px;
    text-align: center;
  }}
  .header-badge .years {{
    font-size: 1.5rem;
    font-weight: 900;
    color: #f5c518;
    line-height: 1;
  }}
  .header-badge .label {{
    font-size: 0.6rem;
    color: #4a7a99;
    letter-spacing: 2px;
    text-transform: uppercase;
    margin-top: 3px;
  }}
  .container {{ padding: 18px 28px; }}
  .section-title {{
    font-size: 0.62rem;
    font-weight: 700;
    letter-spacing: 2.5px;
    color: #2a6080;
    text-transform: uppercase;
    margin: 18px 0 10px 2px;
    padding-left: 8px;
    border-left: 3px solid #1a4a66;
  }}
  .row {{
    display: flex;
    gap: 12px;
    flex-wrap: wrap;
    margin-bottom: 12px;
  }}
  .card {{
    background: #07111a;
    border: 1px solid #0e2233;
    border-radius: 6px;
    overflow: hidden;
    flex: 1;
    min-width: 340px;
  }}
  .card-2 {{ flex: 2; }}
  .card-1 {{ flex: 1; min-width: 300px; }}
  footer {{
    text-align: center;
    padding: 14px;
    color: #1a3a55;
    font-size: 0.68rem;
    letter-spacing: 1px;
    text-transform: uppercase;
    border-top: 1px solid #0a1e2e;
    margin-top: 8px;
  }}
</style>
</head>
<body>

<header>
  <div class="header-left">
    <h1>🚨 Emergencias <span>Colombia</span></h1>
    <p>Inundaciones · Crecientes · Avenidas Torrenciales · UNGRD 1998–2024</p>
  </div>
  <div class="header-badge">
    <div class="years">1998–<br>2024</div>
  </div>
</header>

<div class="container">

  <!-- KPIs -->
  {kpi_html}

  <div class="section-title">Evolución Temporal</div>
  <div class="row">
    <div class="card card-2">{fig_div(fig1)}</div>
    <div class="card card-1">{fig_div(fig2)}</div>
  </div>

  <div class="section-title">Distribución Geográfica</div>
  <div class="row">
    <div class="card card-2">{fig_div(fig3)}</div>
    <div class="card card-2">{fig_div(fig4)}</div>
  </div>

  <div class="section-title">Impacto Humano</div>
  <div class="row">
    <div class="card card-2">{fig_div(fig5)}</div>
    <div class="card card-2">{fig_div(fig6)}</div>
  </div>

  <div class="section-title">Análisis por Departamento</div>
  <div class="row">
    <div class="card card-2">{fig_div(fig7)}</div>
    <div class="card card-2">{fig_div(fig8)}</div>
  </div>

  <div class="section-title">Correlación de Variables</div>
  <div class="row">
    <div class="card card-2">{fig_div(fig9)}</div>
    <div class="card card-2">{fig_div(fig10)}</div>
  </div>

</div>

<footer>Dashboard generado con Python · Plotly · Pandas &nbsp;·&nbsp; Datos UNGRD Colombia 1998–2024</footer>

</body>
</html>
"""

with open(OUTPUT, "w", encoding="utf-8") as f:
    f.write(html_content)

print(f"\n✅  Archivo generado exitosamente:")
print(f"    {OUTPUT}")
print(f"\n    Ábrelo directamente en cualquier navegador (doble clic).\n")

Cargando datos...
  → Separador detectado: ';'
  → Encoding usado: utf-8-sig
  → 17,828 filas cargadas, 45 columnas
  → Columnas detectadas: ['FECHA', 'DEPARTAMENTO', 'MUNICIPIO', 'EVENTO', 'CODIFICACIÓN SEGUN DIVIPOLA', 'MUERTOS', 'HERIDOS', 'DESAPA.', 'PERSONAS', 'FAMILIAS', 'VIV.DESTRU.', 'VIV.AVER.', 'VIAS', 'PTES.VEHIC.', 'PTES.PEAT.', 'ACUED.', 'ALCANT.', 'C. SALUD', 'C.EDUCAT.', 'C.COMUNIT.', 'HECTAREAS', 'OTROS', 'SUBSIDIO DE ARRIENDO', 'ASISTENCIA NO ALIMENTARIA', 'AP.ALIMENT.', 'MATERIALES CONSTRUCCION', 'SACOS - BIGBAG', 'OBRAS DE EMERGENCIA', 'CARROTANQUES - MOTOBOMBAS-PLANTA POTABILIZADORA', 'HORAS MAQUINA RETROEXCAVADORA BULLDOCER INTERVENTORIA', 'APOYO AEREO / TERRESTRE', 'FIC / TRANSFERENCIAS ECONOMICAS', 'INFRAESCTRUCTURA TECNOLOGICA', 'RECURSOS EJECUTADOS', 'OTROS.1', 'No', 'FECHA INICIO', 'FECHA FINAL', 'TRAMITE ANTE UNGRD', 'ATENDIDO', 'NUMERO DE MEMORANDO / OFICIO', 'ANALISIS Y EVALUACION DE LA SOLICITUD', 'DIVIPOLA', 'AÑO', 'MES']
Generando gráficas...
Ensamblando